In [1]:
import csv
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,confusion_matrix
from sklearn.metrics import classification_report, roc_auc_score

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection  import cross_val_score
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from tqdm import tqdm
from joblib import dump, load

In [2]:
data_path = './dataset/classifier_train_test/'
x_train = pd.read_csv(data_path + 'X_train.csv')
x_test = pd.read_csv(data_path + 'X_test.csv')
y_train = pd.read_csv(data_path + 'y_train.csv')
y_test = pd.read_csv(data_path + 'y_test.csv')

In [3]:
scaler = preprocessing.StandardScaler().fit(x_train)
X_train = scaler.transform(x_train)
X_test = scaler.transform(x_test)

y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

In [4]:
param_RF = {'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 250}
model_RF = RandomForestClassifier(**param_RF,random_state=42)

data_path2 = './dataset/classifer_data_5_fold/'
accuracies_RF = []
for i in tqdm(range(5)):
    X_train_fold = pd.read_csv(data_path2 + 'X_train_fold_'+str(i+1)+'.csv')
    X_val_fold = pd.read_csv(data_path2 + 'X_val_fold_'+str(i+1)+'.csv')
    y_train_fold = pd.read_csv(data_path2 + 'y_train_fold_'+str(i+1)+'.csv').values.ravel()
    y_val_fold = pd.read_csv(data_path2 + 'y_val_fold_'+str(i+1)+'.csv').values.ravel()
    
    model_RF.fit(X_train_fold, y_train_fold)
    y_val_fold_RF = model_RF.predict(X_val_fold)
    accuracy = accuracy_score(y_val_fold, y_val_fold_RF)
    accuracies_RF.append(accuracy)
    
average_ACC_RF = np.mean(accuracies_RF)
std_ACC_RF = np.std(accuracies_RF)    

print('The accuracy of cross-validation_RF:',average_ACC_RF,'+/-',std_ACC_RF)

model_RF.fit(X_train, y_train)
y_train_RF = model_RF.predict(X_train)
y_test_RF = model_RF.predict(X_test)

train_AUC_RF = roc_auc_score(y_train, y_train_RF, average='weighted')
train_ACC_RF = accuracy_score(y_train, y_train_RF)
train_Precision_RF = precision_score(y_train, y_train_RF, average='weighted')
train_Recall_RF = recall_score(y_train, y_train_RF, average='weighted')
train_F1_score_RF = f1_score(y_train, y_train_RF, average='weighted')
train_confusion_matrix_RF = confusion_matrix(y_train, y_train_RF)
train_classification_report_RF = classification_report(y_train, y_train_RF, output_dict=True)['weighted avg']
print('___________________________')
print('train_AUC_RF:',train_AUC_RF)
print('train_ACC_RF:', train_ACC_RF)
print('train_Precision_RF:', train_Precision_RF)
print('train_Recall_RF:', train_Recall_RF)
print('train_F1_score_RF:', train_F1_score_RF)
print('train_confusion_matrix_RF:\n', train_confusion_matrix_RF)
print('train_classification_report_RF:\n', train_classification_report_RF)

test_AUC_RF = roc_auc_score(y_test, y_test_RF, average='weighted')
test_ACC_RF = accuracy_score(y_test, y_test_RF)
test_Precision_RF = precision_score(y_test, y_test_RF, average='weighted')
test_Recall_RF = recall_score(y_test, y_test_RF, average='weighted')
test_F1_score_RF = f1_score(y_test, y_test_RF, average='weighted')
test_confusion_matrix_RF = confusion_matrix(y_test, y_test_RF)
test_classification_report_RF = classification_report(y_test, y_test_RF, output_dict=True)['weighted avg']
print('___________________________')
print('test_AUC_RF:',test_AUC_RF)
print('test_ACC_RF:', test_ACC_RF)
print('test_Precision_RF:', test_Precision_RF)
print('test_Recall_RF:', test_Recall_RF)
print('test_F1_score_RF:', test_F1_score_RF)
print('test_confusion_matrix_RF:\n', test_confusion_matrix_RF)
print('test_classification_report_RF:\n', test_classification_report_RF)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:26<00:00, 17.21s/it]


The accuracy of cross-validation_RF: 0.9787591189332385 +/- 0.0034024126751205186
___________________________
train_AUC_RF: 1.0
train_ACC_RF: 1.0
train_Precision_RF: 1.0
train_Recall_RF: 1.0
train_F1_score_RF: 1.0
train_confusion_matrix_RF:
 [[4109    0]
 [   0 4177]]
train_classification_report_RF:
 {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 8286.0}
___________________________
test_AUC_RF: 0.9822896263547672
test_ACC_RF: 0.9826254826254827
test_Precision_RF: 0.98279480148536
test_Recall_RF: 0.9826254826254827
test_F1_score_RF: 0.9826183470336695
test_confusion_matrix_RF:
 [[1062    8]
 [  28  974]]
test_classification_report_RF:
 {'precision': 0.98279480148536, 'recall': 0.9826254826254827, 'f1-score': 0.9826183470336695, 'support': 2072.0}


In [5]:
param_XGB = {'learning_rate': 0.2, 'max_depth': 8, 'min_child_weight': 1, 'n_estimators': 150}
model_XGB = XGBClassifier(**param_XGB,random_state=42)

accuracies_XGB = []
for i in tqdm(range(5)):
    X_train_fold = pd.read_csv(data_path2 + 'X_train_fold_'+str(i+1)+'.csv')
    X_val_fold = pd.read_csv(data_path2 + 'X_val_fold_'+str(i+1)+'.csv')
    y_train_fold = pd.read_csv(data_path2 + 'y_train_fold_'+str(i+1)+'.csv').values.ravel()
    y_val_fold = pd.read_csv(data_path2 + 'y_val_fold_'+str(i+1)+'.csv').values.ravel()
    
    model_XGB.fit(X_train_fold, y_train_fold)
    y_val_fold_XGB = model_XGB.predict(X_val_fold)
    accuracy = accuracy_score(y_val_fold, y_val_fold_XGB)
    accuracies_XGB.append(accuracy)
    
average_ACC_XGB = np.mean(accuracies_XGB)
std_ACC_XGB = np.std(accuracies_XGB)
print('___________________________')
print('The accuracy of cross-validation_XGB:',average_ACC_XGB,'+/-',std_ACC_XGB)

model_XGB.fit(X_train, y_train)
y_train_XGB = model_XGB.predict(X_train)
y_test_XGB = model_XGB.predict(X_test)

train_AUC_XGB = roc_auc_score(y_train, y_train_XGB, average='weighted')
train_ACC_XGB = accuracy_score(y_train, y_train_XGB)
train_Precision_XGB = precision_score(y_train, y_train_XGB, average='weighted')
train_Recall_XGB = recall_score(y_train, y_train_XGB, average='weighted')
train_F1_score_XGB = f1_score(y_train, y_train_XGB, average='weighted')
train_confusion_matrix_XGB = confusion_matrix(y_train, y_train_XGB)
train_classification_report_XGB = classification_report(y_train, y_train_XGB, output_dict=True)['weighted avg']
print('___________________________')
print('train_AUC_XGB:',train_AUC_XGB)
print('train_ACC_XGB:', train_ACC_XGB)
print('train_Precision_XGB:', train_Precision_XGB)
print('train_Recall_XGB:', train_Recall_XGB)
print('train_F1_score_XGB:', train_F1_score_XGB)
print('train_confusion_matrix_XGB:\n', train_confusion_matrix_XGB)
print('train_classification_report_XGB:\n', train_classification_report_XGB)

test_AUC_XGB = roc_auc_score(y_test, y_test_XGB, average='weighted')
test_ACC_XGB = accuracy_score(y_test, y_test_XGB)
test_Precision_XGB = precision_score(y_test, y_test_XGB, average='weighted')
test_Recall_XGB = recall_score(y_test, y_test_XGB, average='weighted')
test_F1_score_XGB = f1_score(y_test, y_test_XGB, average='weighted')
test_confusion_matrix_XGB = confusion_matrix(y_test, y_test_XGB)
test_classification_report_XGB = classification_report(y_test, y_test_XGB, output_dict=True)['weighted avg']
print('___________________________')
print('test_AUC_XGB:',test_AUC_XGB)
print('test_ACC_XGB:', test_ACC_XGB)
print('test_Precision_XGB:', test_Precision_XGB)
print('test_Recall_XGB:', test_Recall_XGB)
print('test_F1_score_XGB:', test_F1_score_XGB)
print('test_confusion_matrix_XGB:\n', test_confusion_matrix_XGB)
print('test_classification_report_XGB:\n', test_classification_report_XGB)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [03:27<00:00, 41.55s/it]


___________________________
The accuracy of cross-validation_XGB: 0.9796043105500443 +/- 0.001721930098574252
___________________________
train_AUC_XGB: 1.0
train_ACC_XGB: 1.0
train_Precision_XGB: 1.0
train_Recall_XGB: 1.0
train_F1_score_XGB: 1.0
train_confusion_matrix_XGB:
 [[4109    0]
 [   0 4177]]
train_classification_report_XGB:
 {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 8286.0}
___________________________
test_AUC_XGB: 0.9819491857406681
test_ACC_XGB: 0.9821428571428571
test_Precision_XGB: 0.9821910809016313
test_Recall_XGB: 0.9821428571428571
test_F1_score_XGB: 0.9821392373473191
test_confusion_matrix_XGB:
 [[1057   13]
 [  24  978]]
test_classification_report_XGB:
 {'precision': 0.9821910809016313, 'recall': 0.9821428571428571, 'f1-score': 0.9821392373473191, 'support': 2072.0}


In [6]:
param_SVC = {'C': 20, 'gamma': 0.0001, 'kernel': 'rbf'}
model_SVC = SVC(**param_SVC,random_state=42)

accuracies_SVC = []
for i in tqdm(range(5)):
    X_train_fold = pd.read_csv(data_path2 + 'X_train_fold_'+str(i+1)+'.csv')
    X_val_fold = pd.read_csv(data_path2 + 'X_val_fold_'+str(i+1)+'.csv')
    y_train_fold = pd.read_csv(data_path2 + 'y_train_fold_'+str(i+1)+'.csv').values.ravel()
    y_val_fold = pd.read_csv(data_path2 + 'y_val_fold_'+str(i+1)+'.csv').values.ravel()
    
    model_SVC.fit(X_train_fold, y_train_fold)
    y_val_fold_SVC = model_SVC.predict(X_val_fold)
    accuracy = accuracy_score(y_val_fold, y_val_fold_SVC)
    accuracies_SVC.append(accuracy)
    
average_ACC_SVC = np.mean(accuracies_SVC)
std_ACC_SVC = np.std(accuracies_SVC)    
print('___________________________')
print('The accuracy of cross-validation_SVC:',average_ACC_SVC,'+/-',std_ACC_SVC)

model_SVC.fit(X_train, y_train)
y_train_SVC = model_SVC.predict(X_train)
y_test_SVC = model_SVC.predict(X_test)

train_AUC_SVC = roc_auc_score(y_train, y_train_SVC, average='weighted')
train_ACC_SVC = accuracy_score(y_train, y_train_SVC)
train_Precision_SVC = precision_score(y_train, y_train_SVC, average='weighted')
train_Recall_SVC = recall_score(y_train, y_train_SVC, average='weighted')
train_F1_score_SVC = f1_score(y_train, y_train_SVC, average='weighted')
train_confusion_matrix_SVC = confusion_matrix(y_train, y_train_SVC)
train_classification_report_SVC = classification_report(y_train, y_train_SVC, output_dict=True)['weighted avg']
print('___________________________')
print('train_AUC_SVC:',train_AUC_SVC)
print('train_ACC_SVC:', train_ACC_SVC)
print('train_Precision_SVC:', train_Precision_SVC)
print('train_Recall_SVC:', train_Recall_SVC)
print('train_F1_score_SVC:', train_F1_score_SVC)
print('train_confusion_matrix_SVC\n:', train_confusion_matrix_SVC)
print('train_classification_report_SVC\n:', train_classification_report_SVC)

test_AUC_SVC = roc_auc_score(y_test, y_test_SVC, average='weighted')
test_ACC_SVC = accuracy_score(y_test, y_test_SVC)
test_Precision_SVC = precision_score(y_test, y_test_SVC, average='weighted')
test_Recall_SVC = recall_score(y_test, y_test_SVC, average='weighted')
test_F1_score_SVC = f1_score(y_test, y_test_SVC, average='weighted')
test_confusion_matrix_SVC = confusion_matrix(y_test, y_test_SVC)
test_classification_report_SVC = classification_report(y_test, y_test_SVC, output_dict=True)['weighted avg']
print('___________________________')
print('test_AUC_SVC:',test_AUC_SVC)
print('test_ACC_SVC:', test_ACC_SVC)
print('test_Precision_SVC:', test_Precision_SVC)
print('test_Recall_SVC:', test_Recall_SVC)
print('test_F1_score_SVC:', test_F1_score_SVC)
print('test_confusion_matrix_SVC:\n', test_confusion_matrix_SVC)
print('test_classification_report_SVC:\n', test_classification_report_SVC)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [02:44<00:00, 32.92s/it]


___________________________
The accuracy of cross-validation_SVC: 0.9723634717064644 +/- 0.003243351045502073
___________________________
train_AUC_SVC: 0.9944500743534471
train_ACC_SVC: 0.9944484672942312
train_Precision_SVC: 0.9944486045219754
train_Recall_SVC: 0.9944484672942312
train_F1_score_SVC: 0.9944484779681658
train_confusion_matrix_SVC
: [[4087   22]
 [  24 4153]]
train_classification_report_SVC
: {'precision': 0.9944486045219754, 'recall': 0.9944484672942312, 'f1-score': 0.9944484779681658, 'support': 8286.0}
___________________________
test_AUC_SVC: 0.9741554274628313
test_ACC_SVC: 0.9744208494208494
test_Precision_SVC: 0.9745083456966976
test_Recall_SVC: 0.9744208494208494
test_F1_score_SVC: 0.9744134196843355
test_confusion_matrix_SVC:
 [[1051   19]
 [  34  968]]
test_classification_report_SVC:
 {'precision': 0.9745083456966976, 'recall': 0.9744208494208494, 'f1-score': 0.9744134196843355, 'support': 2072.0}


In [7]:
model_knn = KNeighborsClassifier(n_neighbors=2,weights='distance',n_jobs=-1)

accuracies_knn = []
for i in tqdm(range(5)):
    X_train_fold = pd.read_csv(data_path2 + 'X_train_fold_'+str(i+1)+'.csv')
    X_val_fold = pd.read_csv(data_path2 + 'X_val_fold_'+str(i+1)+'.csv')
    y_train_fold = pd.read_csv(data_path2 + 'y_train_fold_'+str(i+1)+'.csv').values.ravel()
    y_val_fold = pd.read_csv(data_path2 + 'y_val_fold_'+str(i+1)+'.csv').values.ravel()
    
    model_knn.fit(X_train_fold, y_train_fold)
    y_val_fold_knn = model_knn.predict(X_val_fold)
    accuracy = accuracy_score(y_val_fold, y_val_fold_knn)
    accuracies_knn.append(accuracy)
    
average_ACC_knn = np.mean(accuracies_knn)
std_ACC_knn = np.std(accuracies_knn)    

print('The accuracy of cross-validation_knn:',average_ACC_knn,'+/-',std_ACC_knn)

model_knn.fit(X_train, y_train)
y_train_knn = model_knn.predict(X_train)
y_test_knn = model_knn.predict(X_test)

train_AUC_knn = roc_auc_score(y_train, y_train_knn, average='weighted')
train_ACC_knn = accuracy_score(y_train, y_train_knn)
train_Precision_knn = precision_score(y_train, y_train_knn, average='weighted')
train_Recall_knn = recall_score(y_train, y_train_knn, average='weighted')
train_F1_score_knn = f1_score(y_train, y_train_knn, average='weighted')
train_confusion_matrix_knn = confusion_matrix(y_train, y_train_knn)
train_classification_report_knn = classification_report(y_train, y_train_knn, output_dict=True)['weighted avg']
print('___________________________')
print('train_AUC_knn:',train_AUC_knn)
print('train_ACC_knn:', train_ACC_knn)
print('train_Precision_knn:', train_Precision_knn)
print('train_Recall_knn:', train_Recall_knn)
print('train_F1_score_knn:', train_F1_score_knn)
print('train_confusion_matrix_knn:\n', train_confusion_matrix_knn)
print('train_classification_report_knn:\n', train_classification_report_knn)

test_AUC_knn = roc_auc_score(y_test, y_test_knn, average='weighted')
test_ACC_knn = accuracy_score(y_test, y_test_knn)
test_Precision_knn = precision_score(y_test, y_test_knn, average='weighted')
test_Recall_knn = recall_score(y_test, y_test_knn, average='weighted')
test_F1_score_knn = f1_score(y_test, y_test_knn, average='weighted')
test_confusion_matrix_knn = confusion_matrix(y_test, y_test_knn)
test_classification_report_knn = classification_report(y_test, y_test_knn, output_dict=True)['weighted avg']
print('___________________________')
print('test_AUC_knn:',test_AUC_knn)
print('test_ACC_knn:', test_ACC_knn)
print('test_Precision_knn:', test_Precision_knn)
print('test_Recall_knn:', test_Recall_knn)
print('test_F1_score_knn:', test_F1_score_knn)
print('test_confusion_matrix_knn:\n', test_confusion_matrix_knn)
print('test_classification_report_knn:\n', test_classification_report_knn)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:36<00:00,  7.31s/it]


The accuracy of cross-validation_knn: 0.9663281046960186 +/- 0.005957862798653271
___________________________
train_AUC_knn: 1.0
train_ACC_knn: 1.0
train_Precision_knn: 1.0
train_Recall_knn: 1.0
train_F1_score_knn: 1.0
train_confusion_matrix_knn:
 [[4109    0]
 [   0 4177]]
train_classification_report_knn:
 {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 8286.0}
___________________________
test_AUC_knn: 0.9715335683772642
test_ACC_knn: 0.972007722007722
test_Precision_knn: 0.9723288260922488
test_Recall_knn: 0.972007722007722
test_F1_score_knn: 0.971990158094256
test_confusion_matrix_knn:
 [[1055   15]
 [  43  959]]
test_classification_report_knn:
 {'precision': 0.9723288260922488, 'recall': 0.972007722007722, 'f1-score': 0.971990158094256, 'support': 2072.0}


In [8]:
RF_metrics = {
    'cross-validation_RF_ACC':average_ACC_RF,
    'cross-validation_RF_std':std_ACC_RF,
    'train_ACC_RF': train_ACC_RF,
    'train_AUC_RF': train_AUC_RF,
    'train_Precision_RF': train_Precision_RF,
    'train_Recall_RF':train_Recall_RF,
    'train_F1_score_RF':train_F1_score_RF,
    'train_confusion_matrix_RF': train_confusion_matrix_RF,
    'train_classification_report_RF': train_classification_report_RF,
    'test_ACC_RF': test_ACC_RF,
    'test_AUC_RF': test_AUC_RF,
    'test_Precision_RF': test_Precision_RF,
    'test_Recall_RF':test_Recall_RF,
    'test_F1_score_RF':test_F1_score_RF,
    'test_confusion_matrix_RF': test_confusion_matrix_RF,
    'test_classification_report_RF': test_classification_report_RF
}

output_path = './Result/classifier/'
with open(output_path + 'Classification_RF_metrics_final.txt', 'w') as file:
    for key, value in RF_metrics.items():
        if key in ['train_confusion_matrix_RF', 'train_classification_report_RF',
                   'test_confusion_matrix_RF', 'test_classification_report_RF']:
            file.write(f"{key}:\n{value}\n")
        else:
            file.write(f"{key}: {value}\n")

In [9]:
XGB_metrics = {
    'cross-validation_XGB_ACC':average_ACC_XGB,
    'cross-validation_XGB_std':std_ACC_XGB,
    'train_ACC_XGB': train_ACC_XGB,
    'train_AUC_XGB': train_AUC_XGB,
    'train_Precision_XGB': train_Precision_XGB,
    'train_Recall_XGB':train_Recall_XGB,
    'train_F1_score_XGB':train_F1_score_XGB,
    'train_confusion_matrix_XGB': train_confusion_matrix_XGB,
    'train_classification_report_XGB': train_classification_report_XGB,
    'test_ACC_XGB': test_ACC_XGB,
    'test_AUC_XGB': test_AUC_XGB,
    'test_Precision_XGB': test_Precision_XGB,
    'test_Recall_XGB':test_Recall_XGB,
    'test_F1_score_XGB':test_F1_score_XGB,
    'test_confusion_matrix_XGB': test_confusion_matrix_XGB,
    'test_classification_report_XGB': test_classification_report_XGB
}

with open(output_path + 'Classification_XGB_metrics_final.txt', 'w') as file:
    for key, value in XGB_metrics.items():
        if key in ['train_confusion_matrix_XGB', 'train_classification_report_XGB',
                   'test_confusion_matrix_XGB', 'test_classification_report_XGB']:
            file.write(f"{key}:\n{value}\n")
        else:
            file.write(f"{key}: {value}\n")

In [10]:
SVC_metrics = {
    'cross-validation_SVC_ACC':average_ACC_SVC,
    'cross-validation_SVC_std':std_ACC_SVC,
    'train_ACC_SVC': train_ACC_SVC,
    'train_AUC_SVC': train_AUC_SVC,
    'train_Precision_SVC': train_Precision_SVC,
    'train_Recall_SVC':train_Recall_SVC,
    'train_F1_score_SVC':train_F1_score_SVC,
    'train_confusion_matrix_SVC': train_confusion_matrix_SVC,
    'train_classification_report_SVC': train_classification_report_SVC,
    'test_ACC_SVC': test_ACC_SVC,
    'test_AUC_SVC': test_AUC_SVC,
    'test_Precision_SVC': test_Precision_SVC,
    'test_Recall_SVC':test_Recall_SVC,
    'test_F1_score_SVC':test_F1_score_SVC,
    'test_confusion_matrix_SVC': test_confusion_matrix_SVC,
    'test_classification_report_SVC': test_classification_report_SVC
}

with open(output_path + 'Classification_SVC_metrics_final.txt', 'w') as file:
    for key, value in SVC_metrics.items():
        if key in ['train_confusion_matrix_SVC', 'train_classification_report_SVC',
                   'test_confusion_matrix_SVC', 'test_classification_report_SVC']:
            file.write(f"{key}:\n{value}\n")
        else:
            file.write(f"{key}: {value}\n")

In [11]:
knn_metrics = {
    'cross-validation_knn_ACC':average_ACC_knn,
    'cross-validation_knn_std':std_ACC_knn,
    'train_ACC_knn': train_ACC_knn,
    'train_AUC_knn': train_AUC_knn,
    'train_Precision_knn': train_Precision_knn,
    'train_Recall_knn':train_Recall_knn,
    'train_F1_score_knn':train_F1_score_knn,
    'train_confusion_matrix_knn': train_confusion_matrix_knn,
    'train_classification_report_knn': train_classification_report_knn,
    'test_ACC_knn': test_ACC_knn,
    'test_AUC_knn': test_AUC_knn,
    'test_Precision_knn': test_Precision_knn,
    'test_Recall_knn':test_Recall_knn,
    'test_F1_score_knn':test_F1_score_knn,
    'test_confusion_matrix_knn': test_confusion_matrix_knn,
    'test_classification_report_knn': test_classification_report_knn
}

with open(output_path + 'Classification_knn_metrics.txt', 'w') as file:
    for key, value in knn_metrics.items():
        if key in ['train_confusion_matrix_knn', 'train_classification_report_knn',
                   'test_confusion_matrix_knn', 'test_classification_report_knn']:
            file.write(f"{key}:\n{value}\n")
        else:
            file.write(f"{key}: {value}\n")

In [12]:
np.savetxt(output_path + 'Y_pred_train_RF.csv', y_train_RF, delimiter=',')
np.savetxt(output_path + 'Y_pred_test_RF.csv', y_test_RF, delimiter=',')

np.savetxt(output_path+ 'Y_pred_train_XGB.csv', y_train_XGB, delimiter=',')
np.savetxt(output_path + 'Y_pred_test_XGB.csv', y_test_XGB, delimiter=',')

np.savetxt(output_path + 'Y_pred_train_SVM.csv', y_train_SVC, delimiter=',')
np.savetxt(output_path + 'Y_pred_test_SVM.csv', y_test_SVC, delimiter=',')

np.savetxt(output_path + 'Y_pred_train_KNN.csv', y_train_knn, delimiter=',')
np.savetxt(output_path + 'Y_pred_test_KNN.csv', y_test_knn, delimiter=',')